In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [4]:
stock_symbol = 'AAPL'
start_date = '2020-01-01'
end_date = '2021-01-01'
data = yf.download(stock_symbol, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [5]:
close_prices = data['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
close_prices_scaled = scaler.fit_transform(close_prices)

In [9]:
def create_lstm_data(data, time_steps=1):
    x, y = [], []
    for i in range(len(data) - time_steps):
        x.append(data[i:(i + time_steps), 0])
        y.append(data[i + time_steps, 0])
    return np.array(x), np.array(y)

In [10]:
time_steps = 10
x, y = create_lstm_data(close_prices_scaled, time_steps)
x = np.reshape(x, (x.shape[0], x.shape[1], 1))

In [11]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

/opt/anaconda3/envs/sf-env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
model.fit(x, y, epochs=50, batch_size=32)

Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3232
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0358 
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0261 
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0155 
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0100 
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0079 
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0057 
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0047 
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0044 
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0043 
Epoch 11/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0042 
Epoch 12/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0036 
Epoch 13/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0034 
Epoch 14/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0038 
Epoch 15/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0035 
Epoch 16/50
8/8 ━━━━━━━━━━━━━━━━━━━

In [21]:
future_dates = pd.date_range(start=end_date, periods=30)

# Initialize an array to hold predicted prices
predicted_prices = []

# Start with the last known prices
last_prices = close_prices[-time_steps:]
last_prices_scaled = scaler.transform(last_prices.reshape(-1, 1))
x_pred = np.array([last_prices_scaled[-time_steps:, 0]])
x_pred = np.reshape(x_pred, (x_pred.shape[0], x_pred.shape[1], 1))

# Predict prices for each date in future_dates
for i in range(30):
    predicted_prices_scaled = model.predict(x_pred)
    predicted_price = scaler.inverse_transform(predicted_prices_scaled)
    predicted_prices.append(predicted_price[0, 0])

    # Update x_pred with the new predicted price for the next iteration
    next_input_scaled = scaler.transform(np.array(predicted_price).reshape(-1, 1))
    x_pred = np.roll(x_pred, -1, axis=1)  # Shift the window
    x_pred[0, -1, 0] = next_input_scaled  # Add the new predicted price to the sequence

# Convert predicted_prices list to a numpy array and flatten it
predicted_prices = np.array(predicted_prices)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


/var/folders/ys/2psfql1156jf72wjlnnt4vvc0000gn/T/ipykernel_49130/4051543181.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_pred[0, -1, 0] = next_input_scaled  # Add the new predicted price to the sequence
/var/folders/ys/2psfql1156jf72wjlnnt4vvc0000gn/T/ipykernel_49130/4051543181.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_pred[0, -1, 0] = next_input_scaled  # Add the new predicted price to the sequence
/var/folders/ys/2psfql1156jf72wjlnnt4vvc0000gn/T/ipykernel_49130/4051543181.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


/var/folders/ys/2psfql1156jf72wjlnnt4vvc0000gn/T/ipykernel_49130/4051543181.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_pred[0, -1, 0] = next_input_scaled  # Add the new predicted price to the sequence
/var/folders/ys/2psfql1156jf72wjlnnt4vvc0000gn/T/ipykernel_49130/4051543181.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_pred[0, -1, 0] = next_input_scaled  # Add the new predicted price to the sequence
/var/folders/ys/2psfql1156jf72wjlnnt4vvc0000gn/T/ipykernel_49130/4051543181.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


/var/folders/ys/2psfql1156jf72wjlnnt4vvc0000gn/T/ipykernel_49130/4051543181.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_pred[0, -1, 0] = next_input_scaled  # Add the new predicted price to the sequence
/var/folders/ys/2psfql1156jf72wjlnnt4vvc0000gn/T/ipykernel_49130/4051543181.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_pred[0, -1, 0] = next_input_scaled  # Add the new predicted price to the sequence
/var/folders/ys/2psfql1156jf72wjlnnt4vvc0000gn/T/ipykernel_49130/4051543181.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


/var/folders/ys/2psfql1156jf72wjlnnt4vvc0000gn/T/ipykernel_49130/4051543181.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_pred[0, -1, 0] = next_input_scaled  # Add the new predicted price to the sequence
/var/folders/ys/2psfql1156jf72wjlnnt4vvc0000gn/T/ipykernel_49130/4051543181.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_pred[0, -1, 0] = next_input_scaled  # Add the new predicted price to the sequence
/var/folders/ys/2psfql1156jf72wjlnnt4vvc0000gn/T/ipykernel_49130/4051543181.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single

In [22]:
# Create DataFrame
future_data = pd.DataFrame({'Date': future_dates, 'Predicted Price': predicted_prices.flatten()})

print(future_data)

         Date  Predicted Price
0  2021-01-01       131.371445
1  2021-01-02       131.665085
2  2021-01-03       131.843002
3  2021-01-04       131.846085
4  2021-01-05       131.682968
5  2021-01-06       131.525299
6  2021-01-07       131.326416
7  2021-01-08       131.012268
8  2021-01-09       130.785995
9  2021-01-10       130.603561
10 2021-01-11       130.448273
11 2021-01-12       130.318893
12 2021-01-13       130.167099
13 2021-01-14       130.004547
14 2021-01-15       129.840698
15 2021-01-16       129.681549
16 2021-01-17       129.526703
17 2021-01-18       129.376907
18 2021-01-19       129.234512
19 2021-01-20       129.095825
20 2021-01-21       128.959518
21 2021-01-22       128.825104
22 2021-01-23       128.692276
23 2021-01-24       128.562073
24 2021-01-25       128.434875
25 2021-01-26       128.310608
26 2021-01-27       128.189087
27 2021-01-28       128.070129
28 2021-01-29       127.953629
29 2021-01-30       127.839378
